# DNN

## DNNとは

Deep Learningとは、十分なデータ量があれば、人間の力なしに機械が自動的にデータから特徴を抽出してくれるディープニューラルネットワーク（DNN）を用いた学習のことです。  
DNNは、ニューラルネットワーク（NN）というパターン認識をするように設計された、人間や動物の脳神経回路をモデルとしたアルゴリズムを多層構造化(層を深くする)したもので、昨今注目を浴びています。  
NNを繰り返す繋がりが何度も繰り返されることによって、それは層のように積み重ねられていく。  それが「深層学習」と呼ばれる由来です。(DNNはNNの進化版といった感じです。)  
deep learningはDNNを使って学習する技術であり、つまりdeep learningを使う、ということは「DNNを使って学習する技術を使う」ということです  

しかし層を深くすることによってある問題が発生が発生してしまいます。  
それは**勾配消失問題**です。  
DNNによって層を深くすると、何層も重なってるネットワークでバックプロパゲーション学習の重みを掛けていくと、多重に活性化関数(シグモイド関数)が掛かることになって勾配(誤差)が消失してしまうという問題が発生してしまいます。  
層を遡るに従って誤差が急速に小さくなり 0 になる(あるいは急速に大きくなって爆発する)ために、学習が制御不能に陥ってしまいます。    
また誤差が大きくなり過ぎて爆発してしまうことを**勾配爆発問題**と言います。  
その一つの現れが過学習、すなわち学習サンプルに対する誤差(訓練誤差)はいくらでも小さくできるのに、汎化誤差(サンプルの母集団に対する誤差)を小さくできないことです。  
この勾配消失問題や過学習を克服する手立てが見つかっ たことが，今のディープネットのブームの根底にあります。

# 前回作ったNN

In [39]:
# Package imports
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import sklearn.datasets
import sklearn.linear_model
import matplotlib
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
train = pd.read_csv("train.csv")

In [5]:
target = train["label"]

In [6]:
train = train.drop("label",axis = 1)

In [7]:
# Display plots inline and change default figure size
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10.0, 8.0)

In [8]:
class NN():
    def __init__(self,input_units,nn_units):
        np.random.seed(3)
        input_units = X.shape[1]
        self.w1 = np.random.randn(input_units,nn_units) / np.sqrt(2)
        self.w2 = np.random.randn(nn_units,2) /  np.sqrt(3)
        self.b1 = np.zeros((1,nn_units))
        self.b2 = np.zeros((1,2))
        self.param = { 'w1': self.w1, 'b1': self.b1, 'w2': self.w2, 'b2': self.b2}
        
    def step(self,x):
        if x > 0:
            return 1
        else:
            return 0
    
    def sigmoid(self,x):
        return 1/(1+np.exp(-x))
    
    def relu(self,x):
        return np.maximum(0,x)

    def tanh(self,x):
        e = np.exp(x)
        e_minus = np.exp(-x)
        result = (e-e_minus)/(e+e_minus)
        return result
    
    def softmax(self,a):
        c = np.max(a,axis = 0)
        e_a = np.exp(a)
        e_sum = np.sum(e_a,axis=1, keepdims=True)
        y = e_a/e_sum
        return y
    
    def forward_propagation(self):
        z1 = np.dot(self.x,self.w1) + self.b1
        a1 = self.tanh(z1)
        z2 = np.dot(a1,self.w2) + self.b2
        y = self.softmax(z2)
        return y

    def back_propagation(self,learning_rate=0.01):
        a1 = self.tanh(np.dot(self.x,self.w1) + self.b1)
        delta3 = (self.y_pred-np.identity(2)[self.y])#/len(y)
        delta2 = (1-a1**2) * np.dot(delta3,self.w2.T)
    
        self.w2 -= np.dot(a1.T,delta3)*learning_rate
        self.b2 -= np.sum(delta3,axis=0)*learning_rate
        self.w1 -= np.dot(self.x.T,delta2)*learning_rate
        self.b1 -= np.sum(delta2,axis=0)*learning_rate
        return self.w1,self.w2,self.b1,self.b2
    
    def fit(self,x,y,ite):
        self.x = x
        self.y = y
        self.y_pred = self.forward_propagation()
        
        for i in range(ite):
            
            self.w1,self.w2,self.b1,self.b2 = self.back_propagation()
            self.y_pred = self.forward_propagation()
            self.param = { 'w1': self.w1, 'b1': self.b1, 'w2': self.w2, 'b2': self.b2}
        return self.param
                      
    
    def predict(self,x):
        self.x = x
        pred = self.forward_propagation()
        return np.argmax(pred, axis=1)

# Optimizerの種類

## SGD

SGDとは確率的勾配降下法のことです。  
目的関数が期待値で表された最適化問題に対して有効な最適化アルゴリズムです。
確率的勾配降下法は学習データをシャッフルした上で学習データの中からランダムに1つを取り出して誤差を計算し、パラメーターを更新をします。  
勾配降下法ほどの精度は無いが増えた分だけの学習データのみで再学習する(重みベクトルの初期値は前回の学習結果を流用)ため再学習の計算量が圧倒的に低くなります。  
運が良ければ最急降下法よりも早く最適解にたどり着けますが、運が悪ければいつまで経っても適した答えを導けません。  
### SGDの欠点  
・問題によっては非効率  
・関数の形状が等方的でないと，勾配が最小値を指さない。  

式はこうなります。  
$${\mathbf{w}^{t + 1} \gets \mathbf{w}^{t} - \eta \frac{\partial E(\mathbf{w}^{t})}{\partial \mathbf{w}^{t}}
}$$


## AdaGrad
AdaGrad の基本は SGD ですが、学習パラメータwの各成分ごとに異なるlearning rate を与え、あまり更新されていない成分には高い learning rate を割り振るように工夫します。  

### AdaGradの長所
・ハイパーパラメータが１つだけ  
・シンプルで挙動が分かりやすい  
・学習率は必ず単調減少するので、勾配が極端に変動するものとかでもあまり変な挙動にならない

式はこのようになります
$${\begin{aligned}
    \tilde{v} &\leftarrow \tilde{v} + g_{\theta}^{2} \\
    \theta &\leftarrow \theta - \frac{\alpha}{\sqrt{\tilde{v}} + \epsilon} g_{\theta}
\end{aligned}
}$$


## Adam

Adamは近年最も有力とされている最適化アルゴリズムです。  
Adamでは、これまであまり更新されて来なかったパラメータが優先的に更新されます。  
直感的にMomentumとAdaGradを融合したような手法です。  
2つの手法の移転を組み合わせることで、効率的にパらメーター空間を探索することが期待できます。  
純粋数学的な最適化という観点からすると、これまで最急勾配を選び優先的に下げていく方向だったところに、いや少し待て、それ以外の勾配方向も積極的に試してみようということで、鞍点(多変数実関数の変域の中で、ある方向で見れば極大値だが別の方向で見れば極小値となる点である。)からの抜け出しが速くなります。　　  
数式は難しいですが、このようになります。

$${m_{t+1} = \beta_{1} m_{t} + (1 - \beta_{1}) \nabla E(\mathbf{w}^{t})\\
v_{t+1} = \beta_{2} v_{t} + (1 - \beta_{2}) \nabla E(\mathbf{w}^{t})^{2}\\
\hat{m} = \frac{m_{t+1}}{1 - \beta_{1}^{t}}\\
\hat{v} = \frac{v_{t+1}}{1 - \beta_{2}^{t}}\\
\mathbf{w}^{t+1} = \mathbf{w}^{t} - \alpha \frac{\hat{m}}{\sqrt{\hat{v}} + \epsilon}
}$$




In [9]:
def numerial_gradient(x,y):
    loss_w = lambda w:forward_propagation(x,param)

In [10]:
def SDG(param,grad,learning_rate=0.01):
    for key in param.keys():
        param[key] = learning_rate * grad[key]
    return param

In [11]:
def AdaGrad(param,grad,learning_rate=0.01):
    h = {i:0 for i in param.keys()}
    for key in param.keys():
        h[key] += grad[key] * grad[key]
        param[key] = param[key] - learning_rate *grad[key] /(
            np.sqrt(h[key])+ 1e-7)
    return param,grad

In [12]:
class Adam:

    def __init__(self, lr=0.001, beta1=0.9, beta2=0.999):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.iter = 0
        self.m = None
        self.v = None
        
    def update(self, params, grads):
        if self.m is None:
            self.m, self.v = {}, {}
            for key, val in params.items():
                self.m[key] = np.zeros_like(val)
                self.v[key] = np.zeros_like(val)
        
        self.iter += 1
        lr_t  = self.lr * np.sqrt(1.0 - self.beta2**self.iter) / (1.0 - self.beta1**self.iter)         
        
        for key in params.keys():
            self.m[key] += (1 - self.beta1) * (grads[key] - self.m[key])
            self.v[key] += (1 - self.beta2) * (grads[key]**2 - self.v[key])
            
            params[key] -= lr_t * self.m[key] / (np.sqrt(self.v[key]) + 1e-7)


In [13]:
w1 = np.random.randn(2,3) / np.sqrt(2)
w2 = np.random.randn(3,2) /  np.sqrt(3)
b1 = np.zeros((1,3))
b2 = np.zeros((1,2))
param = { 'w1': w1, 'b1': b1, 'w2': w2, 'b2': b2}
grad = { 'w1': w1, 'b1': b1, 'w2': w2, 'b2': b2}

# 重みの初期化
今までは最初の重みをランダムで生成していましたが、初期値重みの設定の仕方は様々です。  

## Xavier
Xavierは層のノードの数によって、作用させる係数を
変化させます。  
たとえば、前層から渡されるノード数がn個であるときには、標準偏差√nで割rります。  
つまり、初期値のバラツキについては、各層ごとにノードの数で均一化しているイメージになるかと思います。  
このXavierの初期値は「Sigmoid」か「Tanh」に適している初期値として知られています。   
つまり、「ReLU」には最適とはいえません。 


# He
Xavierの初期値と共によく使われる初期値として「Heの初期値」があります。  
作用させる値はXavierと似ていますが、標準偏差√(n/2)で割ります。  
「ReLU」を使う時は、それに適した初期値として、Heの初期値を使います。 

# ガウス分布
左右対称・釣り鐘型の性質をもつ分布として代表的なものが、正規分布（ガウス分布）です。  
その名前（正規分布 normal distribution）からもわかる通り、"normal"な、「ありふれた」「通常の」確率分布です。  
正規分布の最も基本的な性質としては、以下に挙げるものがあります。  
・平均値と最頻値と中央値が一致する。  
・平均値を中心にして左右対称である。  
・分散（標準偏差）が大きくなると、曲線の山は低くなり、左右に広がって平らになる。分散（標準偏差）が小さくなると、山は高くなり、よりとんがった形になる。


In [14]:
def gauss(input_unit,nn_unit,output_unit):
    w1 = np.random.randn(input_unit,nn_unit) * 0.01
    w2 = np.random.randn(nn_unit,output_unit) * 0.01
    b1 = np.zeros((1,nn_unit))
    b2 = np.zeros((1,output_unit))
    param = { 'w1': w1, 'b1': b1, 'w2': w2, 'b2': b2}
    return param
    

In [15]:
def Xavier(input_unit,nn_unit,output_unit):
    w1 = np.random.randn(input_unit,nn_unit) / np.sqrt(input_unit)
    w2 = np.random.randn(nn_unit,output_unit) /  np.sqrt(nn_unit)
    b1 = np.zeros((1,nn_unit))
    b2 = np.zeros((1,output_unit))

In [16]:
def He(input_unit,nn_unit,output_unit):
    w1 = np.random.randn(input_unit,nn_unit) / (np.sqrt(input_unit)*np.sqrt(2))
    w2 = np.random.randn(nn_unit,output_unit) / (np.sqrt(nn_unit)*np.sqrt(2))
    b1 = np.zeros((1,nn_unit))
    b2 = np.zeros((1,output_unit))

# Batch　Normalization

Batch Normalizationは、Deep Learningにおける各重みパラメータを上手くreparametrizationすることで、ネットワークを最適化するための方法の一つです。近年のイノベーションの中でもかなりアツい手法だと紹介されています。  
2015年にIoffe and Szegedyによって発表されました。  
基本的には、各ユニットの出力をミニバッチごとに正則化した新たな値で置き直すことで、内部の変数の分布(内部共変量シフト)が大きく変わるのを防ぎ、学習が早くなる、過学習が抑えられるなどの効果が得られます。  
その効果はかなり大きく、前述の通りDropoutがいらなくなると言われるレベルとのことです。  
簡単に説明すると、各層のアクティベーション分布を強制的に調整する手法です。  
数式はこちらになります。
$${\begin{align}
\mu&=\frac{1}{m}\sum_i z^{(i)} \\
\sigma^2 &= \frac{1}{m}\sum_i (z^{(i)}-\mu)^2 \\
z_{\rm norm}^{(i)} &= \frac{z^{(i)}-\mu}{\sqrt{\sigma^2+\epsilon}} \\
\tilde{z}^{(i)} &= \gamma z_{\rm norm}^{(i)}+\beta
\end{align}
}$$

### 内部共変量シフト
データの分布が訓練時と推定時で異なるような状態のことを言います。  
訓練中にネットワーク内の各層の間で起きる共変量シフトを内部共変量シフトと言うようです。  

### Reparameterization Trick
Reparameterization Trickは変数変換の手法です。  






In [17]:
class bach_norm():    
    def forward(self,X, gamma=0.1, beta=1):
        mu = np.mean(X, axis=0)
        var = np.var(X, axis=0)

        X_norm = (X - mu) / np.sqrt(var + 1e-8)
        out = gamma * X_norm + beta

        cache = (X, X_norm, mu, var, gamma, beta)
        self.cache = cache
        self.out = out

        return out

    def backward(self,dout):
        X, X_norm, mu, var, gamma, beta = self.cache

        N, D = X.shape

        X_mu = X - mu
        std_inv = 1. / np.sqrt(var + 1e-8)

        dX_norm = dout * gamma
        dvar = np.sum(dX_norm * X_mu, axis=0) * -.5 * std_inv**3
        dmu = np.sum(dX_norm * -std_inv, axis=0) + dvar * np.mean(-2. * X_mu, axis=0)

        dX = (dX_norm * std_inv) + (dvar * 2 * X_mu / N) + (dmu / N)
        dgamma = np.sum(dout * X_norm, axis=0)
        dbeta = np.sum(dout, axis=0)

        dX = np.array(dX)
        return dX

    

# Drop Out
Dropoutとは？  
Dropoutとは、ニューラルネットワークの学習時に、一定割合のノードを不活性化させながら学習を行うことで過学習を防ぎ（緩和し）、精度をあげるために手法。   
ニューラルネットワークは訓練データに対するトレース能力に優れており、わりと簡単に過学習を起こしてしまうため、正則化やDropoutのような手法を用いることは重要である。   
学習時に特定のノードを不活性化させて、学習を進めていくことで、過学習を抑えながらパラメーターの更新を行える。


## Drop Outのスクラッチ

In [2]:
class Dropout:

    def __init__(self, dropout_ratio=0.5):
        self.dropout_ratio = dropout_ratio
        self.mask = None

    def forward(self, x, train_flg=True):
        if train_flg:
            self.mask = np.random.rand(*x.shape) > self.dropout_ratio
            return x * self.mask
        else:
            return x * (1.0 - self.dropout_ratio)

    def backward(self, dout):
        return dout * self.mask

# 前回のNNで作った活性化関数

In [18]:
class Relu():
    
    def __init__(self):
        self.mask = None
        
    def forward(self,x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dX = dout
        return dX

In [19]:
class SoftmaxWithLoss():
    def __init__(self):
        self.loss = None
        self.y = None
        
    def softmax(self,a):
        c = np.max(a,axis = 0)
        e_a = np.exp(a-c)
        e_sum = np.sum(e_a,axis=1, keepdims=True)
        y = e_a/e_sum
        return y
    
    def cross_entropy_error(self,y,t):
        delta = 1e-7
        result = -np.sum(t*np.log(y + delta))/len(y)
        return result
        
    def forward(self,x,t):
        self.t = t
        self.y = self.softmax(x)
        self.loss = self.cross_entropy_error(self.y,self.t)
        return self.y
        
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t)/ batch_size      
        return dx

In [20]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = sigmoid(x)
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx


# 活性化関数の前に挟むAffine

In [21]:
class Affine:
    def __init__(self,w,b):
        self.w = w
        self.b = b
        self.x = None
        self.dw = None
        self.db = None
        
        
    def forward(self,x):
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        print(np.argmax(x,axis = 1))
        print("__________________________")        
        self.x = x
        out = np.dot(self.x,self.w) + self.b
        print(np.argmax(out,axis = 1),out.shape)
        print("__________________________")
        return out
    
    def backward(self,dout):
        dx = np.dot(dout,self.w.T)
        self.dw = np.dot(self.x.T,dout)
        self.db = np.sum(dout, axis = 0)
        return dx
        

レイヤーを辞書に入れて値を保存し、バックプロパゲーション時に引き出す。

In [36]:
from collections import OrderedDict
class DNN():
    def __init__(self,layer_node):
        self.masks = []
        np.random.seed(1)
        self.node = layer_node
        self.n = len(layer_node)
        self.wight_params = OrderedDict()
        self.params = {"w" + str(i+1) :np.random.randn(t[0],t[1]) / np.sqrt(t[0]) *np.sqrt(2)
                  for i,t in zip(range(len(layer_node)),layer_node)}
        self.params.update({"b"+ str(i+1):np.zeros((1,t[1]))
                            for i,t in zip(range(len(layer_node)),layer_node)})
        self.layers = OrderedDict()
        self.layers["Affine1"] = Affine(self.params["w1"],self.params["b1"])
        for i in range(self.n-1):
            i += 1
            print(i)
            self.layers["bach_norm"+str(i)] = bach_norm(self.params["w"+str(i)],self.params["b"+str(i)])
            self.layers["Relu"+str(i)] = Relu()
            self.layers["Affine"+str(i+1)] =  Affine(self.params["w"+str(i+1)],self.params["b"+str(i+1)])
        
        self.lastLayers = SoftmaxWithLoss()
    
    def predict(self,x):
        for layer in self.layers.values():
            x = layers.forward(x)
        
        return(x)
    
    def loss(self,x,t):
        y = self.predict(x)
        return self.lastLayers.forward(y,t)
    
    def accuracy(self,x,t):
        y = self.predict(x)
        y = np.argmax(y, axis = 1)
        if t.ndim != 1 : t = np.argmax(t, axis = 1)
            
        accuracy = np.sum(y == t) / float(t, axis = 1)
        return accuracy

    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)    
        
        grads = {}
        for i in range(self.n):
            i += 1
            grads['w'+str(i)] = numerical_gradient(loss_W, self.params['w'+str(i)])
            grads['b'+str(i)] = numerical_gradient(loss_W, self.params['b'+str(i)])
    
    
    def gradient(self,x,t):
        self.loss(x, t)

        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
        
        grads = {}
        for i in range(self.n):
            i += 1
            grads["w"+str(i)] = self.layers["Affine"+str(i)].dw
            grads["b"+str(i)] = self.layers["Affine"+str(i)].db
        
        return grads
            


    def forward_propagation(self,x):
        self.x = x
        self.out = self.x.copy
        for i in range(self.n-1):
            self.w = self.params["w_" + str(i+1)]
            self.b = self.params["b_" + str(i+1)]
            self.out = Affine.forward(self,self.x)
            self.out, cache, mu, var = batchnorm_forward(self.out, 0.1, 1)
            self.x = Relu.forward(self,self.out)
        
        self.w = self.params["w_" + str(self.n)]
        self.b = self.params["b_" + str(self.n)]
        self.out = Affine.forward(self,self.x)
        self.y = softmaxwithLoss.softmax(self,self.out)
        return self.y
    
    def back_propagation(self,t,learning_rate=0.1):
        self.t = np.identity(self.node[-1][-1])[t]
        self.w = self.params["w_" + str(self.n)]
        self.b = self.params["b_" + str(self.n)]
        self.out = softmaxwithLoss.backwarf(self)
        self.out = Affine.backward(self,self.out)
        self.param_change(0)
        
        for i in range(self.n-1):
            self.w = self.params["w_" + str(self.n-i-1)]
            self.b = self.params["b_" + str(self.n-i-1)]
            self.out = Relu.backward(self,self.out,self.n-i-1)
            self.out = Affine.backward(self,self.out)    
            self.param_change(i)
        
            
            
            
    def param_change(self,i,learning_rate = 0.1):
        self.w  -= self.dw * learning_rate
        self.b  -= self.db * learning_rate
        self.params["w_" + str(self.n-i)] = self.w
        self.params["b_" + str(self.n-i)] = self.b
    

ここまででスクラッチは終わりました。　　　　  
実際に別のファイルでこのコードと教科書のコードを少しだけ借りて動かしてみたいと思います。